In [1]:
from decouple import Config, RepositoryEnv
import requests
from json import dumps
import json
import openai
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
import pandas as pd
import logging
from pydantic import BaseModel, ValidationError, Field
from typing import List, Tuple , Union, Optional
from langchain.schema import HumanMessage, SystemMessage, AIMessage
import random
import time

In [2]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff

In [3]:
DOTENV_FILE = 'C:\\Users\\Tiziano Pacifico\\Desktop\\OpenAICourses\\.env'
env_config = Config(RepositoryEnv(DOTENV_FILE))
openai.api_key = env_config.get('CHATGPT_API_KEY')

In [5]:
def create1(messages : List[Union[HumanMessage,SystemMessage,AIMessage]], 
            model_class: BaseModel, retry=2, temperature=0.7, debug = False,
           model="gpt-3.5-turbo-0613") -> BaseModel:
    
    chat = ChatOpenAI(temperature=temperature, model=model)
    
    system_update = f"Please respond ONLY with valid json that conforms to this pydantic\
    json_schema: {model_class.schema_json()}. \
    Do not include additional text other than the object json as we will load this object with json.loads() and pydantic."
    
    system_message = messages[0].content
    update_system_message = system_message + system_update
    messages[0].content = update_system_message
    if debug:
        print(messages)
    last_exception = None
    for i in range(retry+1):
        response = chat(messages)
        #assistant_message= messages
        content = response.content
        try:
            if debug:
                print("------------------------")
                print(response)
            json_content = json.loads(content, strict=False)
        except Exception as e:
            last_exception = e
            error_msg = f"json.loads exception: {e}"
            logging.error(error_msg)
            errorMessage = HumanMessage(content="\nThe previous try generate the following error message:\n###"+error_msg+"###")
            messages.append(errorMessage)
            continue
        try:
            return model_class(**json_content)
        except ValidationError as e:
            last_exception = e
            error_msg = f"pydantic exception: {e}"
            logging.error(error_msg)
            errorMessage = HumanMessage(content="\nThe previous try generate the following error message:\n###"+error_msg+"###")
            messages.append(errorMessage)
    raise last_exception

In [6]:
systemMessage = SystemMessage(content="You are a random topic creator. \
You will create a list of topic for short articles. The number of topic will be given by human message")
humanMessage=HumanMessage(content="Give me a list of 50 topics")
messages = [systemMessage, humanMessage]

In [43]:
class single_topic(BaseModel):
    title: str = Field(description = "The title of the topic")
    description: str = Field(description= "A short description of the topic")
class topics(BaseModel):
    topic: List[single_topic] = Field(description= "A list of single topic, each composed by title and short description")

In [45]:
topics.schema()

C:\Users\Tiziano Pacifico\AppData\Local\Temp\ipykernel_4864\2224441599.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.3/migration/
  topics.schema()


{'$defs': {'single_topic': {'properties': {'title': {'description': 'The title of the topic',
     'title': 'Title',
     'type': 'string'},
    'description': {'description': 'A short description of the topic',
     'title': 'Description',
     'type': 'string'}},
   'required': ['title', 'description'],
   'title': 'single_topic',
   'type': 'object'}},
 'properties': {'topic': {'description': 'A list of single topic, each composed by title and short description',
   'items': {'$ref': '#/$defs/single_topic'},
   'title': 'Topic',
   'type': 'array'}},
 'required': ['topic'],
 'title': 'topics',
 'type': 'object'}

In [8]:
res = create1(messages, topics)

C:\Users\Tiziano Pacifico\AppData\Local\Temp\ipykernel_4864\3382781326.py:6: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.3/migration/
  system_update = f"Please respond ONLY with valid json that conforms to this pydantic json_schema: {model_class.schema_json()}. \


In [9]:
res_dict = res.model_dump()

In [10]:
systemMessage = SystemMessage(content="You are a blog writer. \
You will create a short article for a blog given a topic. The topic will be given in the form enclosed by delimitator ###\
###Topic title: short topic description.### The topic will be given in the following human message. The article has to be short between 500 and 800 token")

In [11]:
class Article(BaseModel):
    text: str = Field(description="The text of the article")

In [12]:
topics = res_dict['topic']
articles = []

In [16]:

chat = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo-0613")
systemMessage = SystemMessage(content="You are a blog writer. \
You will create a short article for a blog given a topic. The topic will be given in the form enclosed by delimitator ###\
###Topic title: short topic description.### The topic will be given in the following human message. The article has to be short between 500 and 800 token")
#article_n = 0
for j in range(3):
    for i in range(20):
        n = random.randint(0, len(topics)-1)
        title = topics[n]['title']
        description = topics[n]['description']
        humanMessage=HumanMessage(content=f"Write a short article about the topic included by the separator ###.\
        ###{title}: {description}###.")
        messages = [systemMessage, humanMessage]
        try: 
            article = chat(messages)
        except Exception as e:
            print("function error")
        else:
            articles.append((article, title, description))
            print(f"Generate article {article_n}")
            article_n +=1
    print("sistem pause")
    time.sleep(300)
    print("sistem restart")



Generate article 240
Generate article 241
Generate article 242
Generate article 243
Generate article 244
Generate article 245
Generate article 246
Generate article 247
Generate article 248
Generate article 249
Generate article 250
Generate article 251
Generate article 252
Generate article 253
Generate article 254
Generate article 255
Generate article 256
Generate article 257
Generate article 258
Generate article 259
sistem pause
sistem restart
Generate article 260
Generate article 261
Generate article 262
Generate article 263
Generate article 264
Generate article 265
Generate article 266
Generate article 267
Generate article 268
Generate article 269
Generate article 270
Generate article 271
Generate article 272
Generate article 273
Generate article 274
Generate article 275
Generate article 276
Generate article 277
Generate article 278
Generate article 279
sistem pause
sistem restart
Generate article 280
Generate article 281
Generate article 282
Generate article 283
Generate article 284

In [17]:
len(articles)

300

In [22]:
articles[1]

(AIMessage(content="###The Benefits of Mindful Eating: Learn how practicing mindful eating can improve digestion and enhance the enjoyment of food.###\n\nIn our fast-paced and hectic lives, we often find ourselves rushing through meals, barely taking the time to savor and enjoy our food. However, practicing mindful eating can offer numerous benefits to our overall well-being.\n\nOne of the key benefits of mindful eating is improved digestion. When we eat mindfully, we are more attuned to our body's hunger and fullness cues. This means that we are less likely to overeat or eat when we are not truly hungry. By eating in this way, we give our digestive system the chance to properly process and break down the food we consume, leading to better digestion and absorption of nutrients.\n\nMindful eating also allows us to fully appreciate and enjoy our food. By paying attention to the taste, texture, and aroma of each bite, we can experience a deeper sense of satisfaction and pleasure from our 

In [19]:
df = pd.DataFrame(articles, columns=["Articolo", "Argomento", "Descrizione"])

In [20]:
df.head()

,Articolo,Argomento,Descrizione
0,"content=""###The Influence of Pop Culture on So...",The Influence of Pop Culture on Society,Explore how popular culture trends influence f...
1,"content=""###The Benefits of Mindful Eating: Le...",The Benefits of Mindful Eating,Learn how practicing mindful eating can improv...
2,"content=""######The Importance of Financial Lit...",The Importance of Financial Literacy,Learn about the significance of understanding ...
3,"content=""###The Impact of Fast Fashion on the ...",The Impact of Fast Fashion on the Environment,Explore the environmental consequences of fast...
4,content='###The Benefits of Regular Exercise f...,The Benefits of Regular Exercise for Mental He...,Learn how physical activity can improve mental...


In [32]:
df['Articolo'] = df['Articolo'].apply(lambda x: x.content.split("\n\n",1)[1])

In [33]:
df.head()

,Articolo,Argomento,Descrizione
0,"Pop culture, short for popular culture, refers...",The Influence of Pop Culture on Society,Explore how popular culture trends influence f...
1,"In our fast-paced and hectic lives, we often f...",The Benefits of Mindful Eating,Learn how practicing mindful eating can improv...
2,Financial literacy is a crucial skill that eve...,The Importance of Financial Literacy,Learn about the significance of understanding ...
3,Fast fashion has become a dominant force in th...,The Impact of Fast Fashion on the Environment,Explore the environmental consequences of fast...
4,Physical exercise has long been known to have ...,The Benefits of Regular Exercise for Mental He...,Learn how physical activity can improve mental...


In [41]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\Applicativi personali\\Test LDA,keybert,bertopic\\Outputs\\"
filename = "Articles_DF.json"

In [40]:
df.to_json(path+filename, orient="records", lines=True)